# Da best heart disease classifier in town
- 13516083 / Abram Perdanaputra
- 13516090 / Timothy Thamrin Andrew Hamonangan Sihombing
- 13516093 / Muhammad Farhan
- 13516153 / Dimas Aditia Pratikto
- 13516155 / Restu Wahyu Kartiko

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt
import numpy as np
import copy
%matplotlib inline

## Read dataset

Now we're going to read the `tubes2_HeartDisease_train` and `tubes2_HeartDisease_test`

In [ ]:
heart_disease = {}
heart_disease['columns_detail'] = [
    'Age', 
    'Sex', 
    'Pain type', 
    'Blood pressure', 
    'Serum cholesterol', 
    'Fasting blood sugar > 120mg/dl', 
    'Resting ECG', 
    'Max heart rate achieved', 
    'exercise induced agina', 
    'ST depression induced by exercise relative to rest', 
    'Peak exercise ST segment', 
    'Number of major vessels colored by flourosopy', 
    'Thal', 
    'Diagnosis'
]
heart_disease['train'] = pd.read_csv('../data/tubes2_HeartDisease_train.csv')
heart_disease['test'] = pd.read_csv('../data/tubes2_HeartDisease_test.csv')

In [ ]:
heart_disease['train'].head()

## Data preprocessing

In [ ]:
# Helpers
def preprocess_data(train_data):
    data = copy.deepcopy(train_data)
    data.Column4 = pd.to_numeric(data.Column4, errors='coerce')
    data.Column5 = pd.to_numeric(data.Column5, errors='coerce')
    data.Column6 = pd.to_numeric(data.Column6, errors='coerce')
    data.Column8 = pd.to_numeric(data.Column8, errors='coerce')
    data.Column9 = pd.to_numeric(data.Column9, errors='coerce')
    data.Column10 = pd.to_numeric(data.Column10, errors='coerce')
    data.Column12 = pd.to_numeric(data.Column12, errors='coerce')
    
    data.loc[data['Column3'] == 1, 'Column3'] = 'typical_agina'
    data.loc[data['Column3'] == 2, 'Column3'] = 'atypical_agina'
    data.loc[data['Column3'] == 3, 'Column3'] = 'non_aginal_pain'
    data.loc[data['Column3'] == 4, 'Column3'] = 'asymtotic'
    
    data.loc[data['Column4'].isnull(), 'Column4'] = data['Column4'].mean()
    data.loc[data['Column5'].isnull(), 'Column5'] = data['Column5'].mean()
    data.loc[data['Column6'].isnull(), 'Column6'] = 0

    data.loc[data['Column7'] == '0', 'Column7'] = 'normal'
    data.loc[data['Column7'] == '1', 'Column7'] = 'having ST-T wave abnormality'
    data.loc[data['Column7'] == '2', 'Column7'] = 'left ventricular hyperthrophy'
    data.loc[data['Column7'] == '?', 'Column7'] = 'normal'

    data.loc[data['Column8'].isnull(), 'Column8'] = 138.348299
    data.loc[data['Column9'].isnull(), 'Column9'] = 0.0
    data.loc[data['Column10'].isnull(), 'Column10'] = 3.937397

    data.loc[data['Column11'] == '1', 'Column11'] = 'upsloping'
    data.loc[data['Column11'] == '2', 'Column11'] = 'flat'
    data.loc[data['Column11'] == '3', 'Column11'] = 'downsloping'
    data.loc[data['Column11'] == '?', 'Column11'] = 'flat'
    
#     data.loc[data['Column12'].isnull(), 'Column12'] = 0.686792

    data.loc[data['Column13'] == '3', 'Column13'] = 'normal'
    data.loc[data['Column13'] == '6', 'Column13'] = 'fixed_defect'
    data.loc[data['Column13'] == '7', 'Column13'] = 'reversable_defect'
    for i, row in data.iterrows():
        if row['Column13'] == '?':
            data.loc[i, 'Column13'] = np.random.choice(['normal', 'reversable_defect', 'fixed_defect'], \
                                               size=1,\
                                               p=[0.46630727762803237, 0.42857142857142855, 0.10512129380053908])[0]
        if np.isnan(row['Column12']):
            data.loc[i, 'Column12'] = np.random.choice([0, 1, 2, 3], \
                                               size=1,\
                                               p=[0.5773584905660377, 0.22264150943396227, 0.13584905660377358, 0.06415094339622641])[0]
    return data

2 jenis:
1. numeric
    - preprocess: normalize
    - null: mean or median, or buat model yang ditrain di datayang ganull, terus predict null
2. categorical
    1. Nominal: urutan gangaruh (fakultas)
         - preprocess: one hot encoding
    2. ordinal: categorical yang urutan ngaruh (rating)
         - preprocess: label encoding pd.get dummies

In [ ]:
heart_disease['train'].replace([np.inf, -np.inf], np.nan)
heart_disease['train'] = preprocess_data(heart_disease['train'])

In [ ]:
heart_disease['train']['Column12'].value_counts().plot.bar()

In [ ]:
heart_disease['train'] = pd.get_dummies(heart_disease['train'])
heart_disease['train'].head()

## Experiment

In [ ]:
cv = 15

### K-Nearest Neighbor

Hyperparameters

In [ ]:
num_neighbor = 5

Preprocess

In [ ]:
def preprocess_KNN(train_data):
    data = copy.deepcopy(train_data)
    data = data.drop(['Column13_normal', 'Column13_fixed_defect', 'Column13_reversable_defect'], axis=1)
    return data
df = preprocess_KNN(heart_disease['train'])
# df = heart_disease['train']

F1 Score

In [ ]:
KNN = KNeighborsClassifier(n_neighbors=num_neighbor, algorithm='ball_tree')
f1_scores = cross_val_score(KNN, df.loc[:, df.columns != 'Column14']\
                , df['Column14'], cv=cv, scoring='f1_micro')
print("F1 Score: {:.4f} +- {:.4f}".format(f1_scores.mean(), f1_scores.std()))

### Naive Bayes

Hyperparameters

Preprocess

F1 Score

### Decision Tree (ID3)

Hyperparameters

Preprocess

F1 Score

### Multilayer Perceptron

Hyperparameters

Preprocess

huyu